In [60]:
import pandas as pd
import re

### В данном файле была проведена очистка полученного датафрейма. Не всегда полученные дефекты можно было очистить регулярными выражениями, поэтому часть работы пришлось выполнить в ручную. Отдельной проблемой стали табличные данные, а также футер сайта.

In [182]:
data = pd.read_csv('Corpus_v4.csv')
data.shape

(32103, 4)

In [183]:
data.columns

Index(['link', 'text', 'pages', 'is_markdown'], dtype='object')

In [82]:
best_inds = []
with open('rows.txt') as f:
    for line in f.readlines():
        best_inds.append(int(line.strip()))

In [184]:
len(best_inds)

20859

In [187]:
data = data.iloc[best_inds]
data

,link,text,pages,is_markdown
0,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False
1,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False
3,https://cbr.ru//reception/,Интернет-приемная Хотите обратиться в Банк Рос...,NaN,False
4,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False
5,https://cbr.ru//news/,Новости Все Новости Пресс-релизы Интервью Выст...,NaN,False
...,...,...,...,...
29235,https://cbr.ru//Collection/Collection/File/270...,Приложение 2. Методическ ие комментарии и поя...,52.0,False
29236,https://cbr.ru//Collection/Collection/File/270...,Приложение 2. Методическ ие комментарии и поя...,53.0,False
29239,https://cbr.ru//Collection/Collection/File/270...,№ 10 окТЯБРЬ 2018 ИнфлЯцИонные ожИданИЯ И поТ...,1.0,False
29240,https://cbr.ru//Collection/Collection/File/270...,окТЯБРЬ 2018 № 10 (22)ИнфлЯцИонные ожИданИЯ И...,2.0,False


In [188]:
data.dropna(subset=['text'], inplace=True)
data.drop_duplicates(subset='text', inplace=True)
data = data[data['text'].str.strip() != '']
data = data[data['text'].apply(len) >= 100]
data.shape

(18839, 4)

In [299]:
# ручной перебор маркдаун файлов
with open('md.txt', 'w') as f:
    for index, row in data[data.is_markdown].iterrows():
        f.write(f'{index} \n')
        f.write(f'{row[1]} \n')

In [120]:
data[data.is_markdown]

,link,text,pages,is_markdown
164,https://cbr.ru//Queries/UniDbQuery/DownloadExc...,[MARKDOWN]| part_number | DT | cname | sname |...,NaN,True
1176,https://cbr.ru//vfs/credit/fzlist/fz1121list-2...,[MARKDOWN]| Перечень | |---| | кредитных орган...,NaN,True
1177,https://cbr.ru//vfs/credit/fzlist/fz1121list-2...,[MARKDOWN]| Перечень | |---| | кредитных орган...,NaN,True
1178,https://cbr.ru//vfs/credit/fzlist/fz1121list-2...,[MARKDOWN]| Перечень | |---| | кредитных орган...,NaN,True
1179,https://cbr.ru//vfs/credit/fzlist/fz1121list-2...,[MARKDOWN]| Перечень | |---| | кредитных орган...,NaN,True
...,...,...,...,...
2706,https://cbr.ru//Content/Document/File/156398/d...,[MARKDOWN]| Динамические ряды основных показат...,NaN,True
2709,https://cbr.ru//Content/Document/File/156330/d...,[MARKDOWN]| Динамические ряды основных показат...,NaN,True
2711,https://cbr.ru//vfs/analytics/finflows/stat_fi...,[MARKDOWN]| Статистическое приложение к публик...,NaN,True
2886,https://cbr.ru//Content/Document/File/70583/re...,[MARKDOWN]| Приложение 2 | |---| | Нормативы о...,NaN,True


In [189]:
bad_ind = [1971, 1972, 1973, 1974, 1975, 1976, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2035, 
           2084, 2088, 2090, 2092, 2093, 2094, 
           2106, 2107, 2108, 2109, 2142, 
           2522, 2523, 2524, 2525, 2526, 2527, 
           2529, 2530, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 
           2544, 2545, 2546, 2547, 2548, 2549, 2550, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2677, 2686, 2709, 2886]

In [190]:
data2 = data.copy()

In [191]:
import re

# Предположим, что data2 - это ваш DataFrame, а bad_ind - это индекс строки

# Функция для замены цифр на пустоту в строке
def remove_numbers(text):
    return re.sub(r'\d', '', text)

# Применяем функцию ко всем полям 'text' в DataFrame data2 по индексу bad_ind
data2.loc[bad_ind]['text'] = data2.loc[bad_ind]['text'].apply(remove_numbers)

In [345]:
# ручной перебор обычных файлов
with open('common_texts.txt', 'w') as f:
    for index, row in data2[~data2.is_markdown].iterrows():
        f.write(f'{index} \n')
        f.write(f'{row[1]} \n')

In [ ]:
import re

# Функция для удаления последовательностей точек и тире
def remove_dots_and_dashes(text):
    # Удаление последовательностей точек
    text = re.sub(r'\.\s*\.\s*\.\s*', '', text)
    # Удаление последовательностей тире
    text = re.sub(r'–\s*–\s*–\s*', '', text)
    return text

# Применение функции к столбцу 'text'
data2['text'] = data2['text'].apply(remove_dots_and_dashes)


In [ ]:
range(8880, 21263)

In [249]:
bad_com_texts= [21265, 21266, 21269, 21270, 21273, 21274]

In [253]:
data2.head()

,link,text,pages,is_markdown,is_table
0,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False,False
3,https://cbr.ru//reception/,Интернет-приемная Хотите обратиться в Банк Рос...,NaN,False,False
6,https://cbr.ru//rbr/,Решения Банка России Рубрикатор Обновлено 07.0...,NaN,False,False
7,https://cbr.ru//contacts/,Контактная информация Контактный центр В насто...,NaN,False,False
8,https://cbr.ru//sitemap,О сайте Карта сайта Денежно-кредитная политика...,NaN,False,False


In [266]:
data2.loc[8880:21263]

,link,text,pages,is_markdown,is_table
8880,https://cbr.ru//Collection/Collection/File/252...,Таблица 1 Показатель 1.01.14 1.01.15 1.01.16 ...,2.0,False,True
8882,https://cbr.ru//Collection/Collection/File/252...,Таблица 3 2007 2008 2009 2010 2011 2012 2013 ...,4.0,False,False
8886,https://cbr.ru//Collection/Collection/File/252...,Таблица 7 Концентрация активов по банковскому...,8.0,False,False
8943,https://cbr.ru//Collection/Collection/File/252...,Таблица 56 Валютные активы и пассивы в совоку...,65.0,False,False
8949,https://cbr.ru//Collection/Collection/File/252...,Таблица 62 Распределение кредитных организаци...,71.0,False,False
...,...,...,...,...,...
21258,https://cbr.ru//Collection/Collection/File/409...,Средства на счетах эскроу млрд руб. / % 175% ...,3.0,False,False
21259,https://cbr.ru//Collection/Collection/File/409...,Основные показатели проектного финансирования...,4.0,False,False
21261,https://cbr.ru//Collection/Collection/File/398...,1 005 1 301 1 6752 232 2 5792 7243 4174 1505 ...,2.0,False,False
21262,https://cbr.ru//Collection/Collection/File/398...,Средства на счетах эскроу млрд руб. 191% 118%...,3.0,False,False


In [161]:
data2.loc[244]

link           https://cbr.ru//Queries/UniDbQuery/File/90134/...
text           9. HacT051IIJ,ee YKa3aHHe BCTyrraeT B CHJIY rr...
pages                                                        NaN
is_markdown                                                False
Name: 244, dtype: object

In [169]:
244 in list(map(int, data2.index))

True

In [212]:
bad_com_texts_red = []

for i in bad_com_texts:
    if i in list(map(int, data2.index)):
        bad_com_texts_red.append(i)
print(bad_com_texts_red)

[4155, 5841, 5843, 5847, 6375, 6419, 6534, 6578, 7951]


In [154]:
print(*list(range(2563, 2569+ 1)), sep=', ')

2563, 2564, 2565, 2566, 2567, 2568, 2569


In [192]:
filter = data2['text'].str.contains("Пн Вт Ср Чт Пт Сб Вс 1 2 3 4 5 6 7 8 9 10 11 12 13")
data2 = data2[~filter]
data2.shape

(18744, 4)

In [194]:
filter = data2['text'].str.contains("2024 г. 2023 г. 2022 г. 2021 г. 2020 г. 2019 г.")
data2 = data2[~filter]
data2.shape

(18672, 4)

In [195]:
filter = data2['text'].str.contains("2011 2010 2009 2008 2007 2006 2005 2004 2003 2002 2001")
data2 = data2[~filter]
data2.shape

(18643, 4)

In [196]:
filter = data2['text'].str.contains("Январь Декабрь Ноябрь Октябрь Сентябрь Август Июль Июнь")
data2 = data2[~filter]
data2.shape

(18598, 4)

In [197]:
filter = data2['text'].str.contains("май 2015 г.: апрель 2015 г.: март 2015 г.: февраль 2015 г.: январь 2014 г.")
data2 = data2[~filter]
data2.shape

(18546, 4)

In [203]:
filter = data2['text'].str.contains("2023 г. 2022 г. 2021 г. 2020 г.")
data2 = data2[~filter]
data2.shape

(18442, 4)

In [205]:
filter = data2['text'].str.contains("2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026")
data2 = data2[~filter]
data2.shape

(18438, 4)

In [209]:
import re

pattern = re.compile(r'^[\d\s]+')

# Функция для удаления цифр и пробелов до первого слова в строке
def remove_digits_and_spaces(text):
    return pattern.sub('', text)

data2.loc[8014:8314, 'text'] = data2.loc[8014:8314, 'text'].apply(remove_digits_and_spaces)

In [226]:
data2['text'] = data2['text'].str.replace('\x07', '')

In [247]:
filter = data2['text'].str.contains("Материал подготовлен Департаментом статистики. При использовании материалов выпуска ссылка на Банк России обязательна.")
data2 = data2[~filter]
data2.shape

(18366, 4)

In [198]:
data2 = data2.drop([244, 245, 308, 1135, 1940, 2347, 2359, 2363, 2386, 2387, 2388, 2435, 2437, 2476, 2563, 2565, 2566, 2568, 2569, 2696])
data2.shape

(18526, 4)

In [200]:
filter = data2['text'].str.contains("За все время За последний месяц За последний год")
data2 = data2[~filter]
data2.shape

(18461, 4)

In [263]:
table_inds = []
for i in range(8880, 21264+1):
    if i in data2.loc[8880:21264].index:
        table_inds.append(i)
len(table_inds)

7665

In [297]:
import re


def remove_numbers_commas_periods(text):
    cleaned_text = re.sub(r'[\d,.%–<≤>/]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

# Применяем функцию ко всем полям 'text' в DataFrame data2 по индексу bad_ind
data2.loc[table_inds, 'text'] = data2.loc[table_inds, 'text'].apply(remove_numbers_commas_periods)
data2.loc[table_inds, 'is_table'] = True

In [296]:
data2.loc[table_inds, 'text'] = data2.loc[table_inds, 'text'].apply(remove_numbers_commas_periods)


In [298]:
data2.loc[table_inds]

,link,text,pages,is_markdown,is_table
8880,https://cbr.ru//Collection/Collection/File/252...,Таблица Показатель Совокупные активы (пассивы...,2.0,False,True
8882,https://cbr.ru//Collection/Collection/File/252...,Таблица Активы - Капитал Кредиты предоставлен...,4.0,False,True
8886,https://cbr.ru//Collection/Collection/File/252...,Таблица Концентрация активов по банковскому с...,8.0,False,True
8943,https://cbr.ru//Collection/Collection/File/252...,Таблица Валютные активы и пассивы в совокупны...,65.0,False,True
8949,https://cbr.ru//Collection/Collection/File/252...,Таблица Распределение кредитных организаций п...,71.0,False,True
...,...,...,...,...,...
21258,https://cbr.ru//Collection/Collection/File/409...,Средства на счетах эскроу млрд руб Высокие пр...,3.0,False,True
21259,https://cbr.ru//Collection/Collection/File/409...,Основные показатели проектного финансирования...,4.0,False,True
21261,https://cbr.ru//Collection/Collection/File/398...,к к к к кНеиспользованный кредитный лимит Зад...,2.0,False,True
21262,https://cbr.ru//Collection/Collection/File/398...,Средства на счетах эскроу млрд руб Рекордные ...,3.0,False,True


In [333]:
# пришлось визуально выделить дефектные строки..
to_clear = list(range(25772, 25814 + 1)) + list(range(25847, 25889 + 1)) + list(range(25919, 25961 + 1))
to_clear += list(range(25989, 26031 + 1)) + list(range(26060, 26102 + 1)) + list(range(26128 , 26170 + 1))
to_clear += list(range(26196, 26238 + 1)) + list(range(26268, 26310 + 1)) + list(range(26339, 26381 + 1))
to_clear += list(range(26405, 26447 + 1)) + list(range(26472, 26512 + 1)) + list(range(26534, 26576 + 1))

to_clear += list(range(26602, 26643 + 1)) + list(range(26666, 26708 + 1)) + list(range(26735, 26777 + 1))
to_clear += list(range(26800, 26842 + 1)) + list(range(26862, 26903 + 1)) + list(range(26928, 26968 + 1))
to_clear += list(range(26990, 27032 + 1)) + list(range(27054, 27095 + 1)) + list(range(27115, 27156 + 1))

to_clear += list(range(27174, 27216 + 1)) + list(range(27236, 27278 + 1)) + list(range(27303, 27345 + 1))
to_clear += list(range(27365, 27407 + 1)) + list(range(27425, 27467 + 1)) + list(range(27783, 27824 + 1))
to_clear += list(range(27841, 27882 + 1)) + list(range(27900, 27941 + 1)) + list(range(27964, 28004 + 1))

to_clear += list(range(28021, 28063 + 1)) + list(range(28140, 28182 + 1)) + list(range(28198, 28240 + 1))
to_clear += list(range(28255, 28297 + 1)) + list(range(28312, 28354 + 1)) + list(range(28369, 28411 + 1))
to_clear += list(range(28427, 28469 + 1)) + list(range(28486, 28527 + 1)) + list(range(28543, 28585 + 1))

to_clear += list(range(28602, 28626 + 1)) + list(range(28648, 28667 + 1)) + list(range(28701, 28702 + 1))
to_clear += list(range(28716, 28734 + 1)) + list(range(29139, 29227 + 1))

to_clear = [x for x in to_clear if x in data2.index]
len(to_clear)

1791

In [334]:
import re

# Функция для удаления чисел
def remove_numbers(text):
    return re.sub(r'\d', '', text)

# Функция для удаления последовательностей букв длиной 3
def remove_3_letter_sequences(text):
    return re.sub(r'\b\w{3}\b', '', text)

# Функция для удаления длинных пробелов
def remove_long_spaces(text):
    return re.sub(r'\s{2,}', ' ', text)

# Применение функций к столбцу 'text'
data2.loc[to_clear, 'text'] = (
    data2.loc[to_clear, 'text']
    .apply(remove_numbers)
    .apply(remove_3_letter_sequences)
    .apply(remove_long_spaces)
)


In [340]:
data2.loc[to_clear, 'text'] = data2.loc[to_clear, 'text'].apply(remove_numbers).apply(remove_3_letter_sequences).apply(remove_long_spaces)

In [343]:
data2.loc[to_clear, 'is_table'] = True

In [305]:
data2 = data2.drop([29214])
data2.shape

(18356, 5)

In [344]:
data2.to_csv('data_final.csv', index=None)